In [ ]:
import os
import numpy as np
from osgeo import gdal, gdal_array
import math


In [ ]:
def apply_formula(data, formula_id):
    """
    根据formula_id应用不同的公式计算新的栅格值。
    """
    # 使用np.where避免除以0的错误
    H = np.where(data == 0, 1, data)  # 防止除以0
    
    if formula_id == 0:
        A = 21.4927 - 14.4971 * np.log(4.2971 - 1.3091 * np.log(H))
    elif formula_id == 1:
        A = 21.2355 - 16.245 * np.log(4.2410 - 1.2931 * np.log(H))
    elif formula_id == 2:
        A = 39.0843 - 11.5511 * np.log(318.3676 / (12.85630 * H - 18.2439) - 1)
    elif formula_id == 3:
        A = 39.5571 - 12.1033 * np.log(333.3676 / (12.8437 * H - 15.3651) - 1)
    elif formula_id == 4:
        A = 40.5136 - 28.9032 * np.log(4.1591 - 1.2134 * np.log(H))
    elif formula_id == 5:
        A = 42.8467 - 25.7542 * np.log(4.7298 - 1.376 * np.log(H))
    elif formula_id == 6:
        A = 32.5028 - 7.4971 * np.log(81.7489 * H ^(- 1.3438) - 1)
    elif formula_id == 7:
        A = 34.4237 - 7.8914 * np.log(81.6593 * H ^(- 1.3458) - 1)
    else:
        raise ValueError("Invalid formula_id.")
    
    # 处理后的值再次将原始为0的位置置为0
    A[data == 0] = 0
    return A

In [ ]:
def process_raster(file_path, formula_id):
    """
    处理单个栅格数据文件，并返回处理后的数组。
    """
    dataset = gdal.Open(file_path)
    band = dataset.GetRasterBand(1)
    data = band.ReadAsArray().astype(float)
    
    # 应用计算公式
    processed_data = np.round(apply_formula(data, formula_id))
    
    return processed_data, dataset.GetGeoTransform(), dataset.GetProjection()

def merge_rasters(processed_data_list, geo_transform, projection, output_path):
    """
    合并多个栅格数据为一个，并保存到指定路径。
    """
    # 假设所有栅格尺寸相同，以第一个栅格的尺寸为准
    rows, cols = processed_data_list[0].shape
    driver = gdal.GetDriverByName('GTiff')
    out_raster = driver.Create(output_path, cols, rows, 1, gdal.GDT_Float32)
    out_raster.SetGeoTransform(geo_transform)
    out_raster.SetProjection(projection)
    
    # 初始化一个与单个栅格相同大小的数组，用于累加数据
    merged_data = np.zeros((rows, cols), dtype=np.float32)
    for data in processed_data_list:
        merged_data += data
    
    out_band = out_raster.GetRasterBand(1)
    out_band.WriteArray(merged_data)
    out_band.FlushCache()

def main(folder_path, output_path):
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')]
    processed_data_list = []
    for i, file_path in enumerate(files):
        processed_data, geo_transform, projection = process_raster(file_path, i % 8)
        processed_data_list.append(processed_data)
    
    # 合并栅格并保存
    merge_rasters(processed_data_list, geo_transform, projection, output_path)

In [ ]:
if __name__ == '__main__':
    folder_path = os.path.join("data", "input")  # 输入路径：
    output_path = os.path.join("data", "output")  # 输出路径：
    main(folder_path, output_path)


In [ ]:

# 打开栅格数据文件
dataset = gdal.Open(os.path.join("data", "output"), gdal.GA_ReadOnly)

# 读取栅格数据的第一个波段
band = dataset.GetRasterBand(1)
raster_data = band.ReadAsArray()

# 对数据进行四舍五入取整
rounded_data = np.round(raster_data).astype(int)

# 获取原始栅格的一些参数以用于创建新的栅格文件
geotransform = dataset.GetGeoTransform()
projection = dataset.GetProjection()
[nrows, ncols] = raster_data.shape

# 创建新的栅格文件
driver = gdal.GetDriverByName('GTiff')
new_dataset = driver.Create(os.path.join("data", "output"), ncols, nrows, 1, gdal.GDT_Int32)

# 设置地理变换和投影
new_dataset.SetGeoTransform(geotransform)
new_dataset.SetProjection(projection)

# 将四舍五入后的数据写入新的栅格文件
new_dataset.GetRasterBand(1).WriteArray(rounded_data)

# 保存并关闭数据集
new_dataset.FlushCache()
new_dataset = None

In [ ]:


# 打开栅格A和B
dataset_a = gdal.Open(os.path.join("data", "input"), gdal.GA_ReadOnly)
dataset_b = gdal.Open(os.path.join("data", "input"), gdal.GA_ReadOnly)

# 假设栅格A和B已经是相同的分辨率和坐标系
# 读取栅格数据
band_a = dataset_a.GetRasterBand(1)
band_b = dataset_b.GetRasterBand(1)
data_a = band_a.ReadAsArray()
data_b = band_b.ReadAsArray()

# 这里简化处理，假设A和B完全重叠
# 在实际情况中，你需要根据地理位置计算重叠区域，并相应地调整数组索引
data_a = data_b

# 获取栅格A的一些参数以用于保存结果
geotransform = dataset_a.GetGeoTransform()
projection = dataset_a.GetProjection()
[nrows, ncols] = data_a.shape

# 创建新的栅格文件用于保存结果
driver = gdal.GetDriverByName('GTiff')
new_dataset = driver.Create(os.path.join("data", "output"), ncols, nrows, 1, gdal.GDT_Float32)

# 设置地理变换和投影
new_dataset.SetGeoTransform(geotransform)
new_dataset.SetProjection(projection)

# 将修改后的数据写入新的栅格文件
new_dataset.GetRasterBand(1).WriteArray(data_a)

# 保存并关闭数据集
new_dataset.FlushCache()
new_dataset = None


In [ ]:


# 打开原始栅格文件
src_filename = os.path.join("data", "input")
src_ds = gdal.Open(src_filename, gdal.GA_ReadOnly)

# 获取栅格的第一个波段
src_band = src_ds.GetRasterBand(1)

# 将无数据值设置为2023（如果原始数据已经设置了无数据值，这一步可以跳过）
src_band.SetNoDataValue(2023)

# 读取栅格数据
data = src_band.ReadAsArray()

# 将无数据值2023更改为0
data[data == 2023] = 0

# 创建输出栅格文件
driver = gdal.GetDriverByName('GTiff')
dst_filename = os.path.join("data", "output")
dst_ds = driver.Create(dst_filename, src_ds.RasterXSize, src_ds.RasterYSize, 1, src_band.DataType)

# 将原始栅格的地理变换和投影复制到新栅格
dst_ds.SetGeoTransform(src_ds.GetGeoTransform())
dst_ds.SetProjection(src_ds.GetProjection())

# 获取新栅格的第一个波段并写入数据
dst_band = dst_ds.GetRasterBand(1)
dst_band.WriteArray(data)

# 设置新的无数据值为0
dst_band.SetNoDataValue(0)

# 清理
dst_band = None
dst_ds = None
src_band = None
src_ds = None

print("完成栅格无数据值的更改。")


In [ ]:


# 打开栅格文件A和B
dataset_a = gdal.Open(os.path.join("data", "input"), gdal.GA_ReadOnly)
dataset_b = gdal.Open('/path/to/your/raster/B.tif', gdal.GA_ReadOnly)

# 读取各自的栅格波段（这里假设我们关心的是第一个波段）
band_a = dataset_a.GetRasterBand(1)
band_b = dataset_b.GetRasterBand(1)

# 读取栅格数据为数组
data_a = band_a.ReadAsArray()
data_b = band_b.ReadAsArray()

# 获取B的无数据值，这里假设已知或通过band_b.GetNoDataValue()获取
no_data_value_b = band_b.GetNoDataValue()

# 如果B中有无数据值定义，则使用它来识别无数据的像元；否则，可能需要设定一个特定的值或条件
if no_data_value_b is not None:
    mask = data_b == no_data_value_b
else:
    # 假设无数据像元可以通过一个条件来识别，例如NaN
    mask = np.isnan(data_b)

# 只在B有数据的地方替换A的值
data_a[~mask] = data_b[~mask]

# 保存结果到新的栅格文件
driver = dataset_a.GetDriver()
out_dataset = driver.Create('/path/to/your/merged_raster.tif', dataset_a.RasterXSize, dataset_a.RasterYSize, 1, band_a.DataType)
out_band = out_dataset.GetRasterBand(1)

# 设置无数据值，如果有必要
if band_a.GetNoDataValue() is not None:
    out_band.SetNoDataValue(band_a.GetNoDataValue())

out_band.WriteArray(data_a)

# 设置地理变换和投影信息
out_dataset.SetGeoTransform(dataset_a.GetGeoTransform())
out_dataset.SetProjection(dataset_a.GetProjection())

# 清理
out_band.FlushCache()
out_dataset = None


In [ ]:

# 打开原始栅格文件
src_filename = r'D:\1毕业相关\二、数据整理\树种坡向因子\非扰动取整.tif'
src_ds = gdal.Open(src_filename, gdal.GA_ReadOnly)
if src_ds is None:
    raise FileNotFoundError(f"文件 {src_filename} 未找到。")

# 读取栅格数据为数组
band = src_ds.GetRasterBand(1)  # 假设我们只处理第一个波段
arr = band.ReadAsArray()

# 将nan值替换为0
arr = np.nan_to_num(arr, nan=0)

# 创建新栅格文件
driver = gdal.GetDriverByName('GTiff')
dst_filename = r'D:\1毕业相关\二、数据整理\非扰动年龄.tif'
dst_ds = driver.Create(dst_filename, src_ds.RasterXSize, src_ds.RasterYSize, 1, band.DataType)
dst_ds.SetGeoTransform(src_ds.GetGeoTransform())
dst_ds.SetProjection(src_ds.GetProjection())

# 写入处理后的数据
dst_band = dst_ds.GetRasterBand(1)
dst_band.WriteArray(arr)

# 清理
dst_band = None
dst_ds = None
src_ds = None


In [ ]:


def set_nodata_to_zero(input_folder, output_folder):
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 遍历输入文件夹中的所有文件
    for filename in os.listdir(input_folder):
        if filename.endswith('.tif'):  # 假设栅格文件为TIFF格式
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # 打开栅格文件
            dataset = gdal.Open(input_path, gdal.GA_ReadOnly)
            band = dataset.GetRasterBand(1)  # 假设只处理第一个波段

            # 读取栅格数据
            data = band.ReadAsArray()

            # 将无数据值设置为0
            nodata_value = band.GetNoDataValue()
            if nodata_value is not None:
                data[data == nodata_value] = 0

            # 创建新的栅格文件
            driver = gdal.GetDriverByName('GTiff')
            out_dataset = driver.Create(output_path, dataset.RasterXSize, dataset.RasterYSize, 1, band.DataType)
            out_dataset.SetGeoTransform(dataset.GetGeoTransform())
            out_dataset.SetProjection(dataset.GetProjection())
            out_band = out_dataset.GetRasterBand(1)
            out_band.WriteArray(data)

            # 设置新的无数据值（如果需要）
            # out_band.SetNoDataValue(0)

            # 清理
            out_band.FlushCache()
            out_dataset = None
            dataset = None

            print(f'Processed {input_path} and saved to {output_path}')

# 设置输入和输出文件夹路径
input_folder = os.path.join("data", "input")
output_folder = os.path.join("data", "output")

set_nodata_to_zero(input_folder, output_folder)


In [ ]:


def merge_rasters(folder_path, output_path):
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')]
    if not files:
        print("No raster files found.")
        return
    
    # 读取第一个文件以获取基础信息
    ref_raster = gdal.Open(files[0])
    geo_transform = ref_raster.GetGeoTransform()
    projection = ref_raster.GetProjection()
    x_size = ref_raster.RasterXSize
    y_size = ref_raster.RasterYSize
    dtype = ref_raster.GetRasterBand(1).DataType
    ref_raster = None  # 关闭文件
    
    # 初始化累加数组
    accum_array = np.zeros((y_size, x_size), dtype=float)
    
    # 累加栅格值
    for file in files:
        raster = gdal.Open(file)
        array = raster.ReadAsArray()
        accum_array += array
        raster = None  # 关闭文件
    
    # 创建新的栅格文件
    driver = gdal.GetDriverByName('GTiff')
    out_raster = driver.Create(output_path, x_size, y_size, 1, dtype)
    out_raster.SetGeoTransform(geo_transform)
    out_raster.SetProjection(projection)
    out_band = out_raster.GetRasterBand(1)
    out_band.WriteArray(accum_array)
    
    # 清理
    out_band.FlushCache()
    out_raster = None

# 使用函数
folder_path = os.path.join("data", "input")
output_path = os.path.join("data", "output")
merge_rasters(folder_path, output_path)


In [ ]:

# 打开原始栅格文件
input_raster = os.path.join("data", "input")  # 修改为你的输入文件路径
ds = gdal.Open(input_raster)
band = ds.GetRasterBand(1)
nodata = band.GetNoDataValue()

# 读取栅格数据
data = band.ReadAsArray()

# 应用公式，只对非空值进行计算
# 注意：这里假设无数据值已经被设置为了np.nan或者一个特定的无数据标记
# 如果无数据值不是np.nan，你需要先将它们转换为np.nan或者一个适当的值
with np.errstate(divide='ignore', invalid='ignore'):  # 忽略可能的数学错误
    A = np.where(data == nodata, nodata, 21.4927 - 14.4971 * np.log(4.2971 - 1.3091 * np.log(data)))

# 创建新栅格
driver = gdal.GetDriverByName('GTiff')
output_raster = os.path.join("data", "input")  # 修改为你的输出文件路径
out_ds = driver.Create(output_raster, ds.RasterXSize, ds.RasterYSize, 1, band.DataType)
out_ds.SetGeoTransform(ds.GetGeoTransform())
out_ds.SetProjection(ds.GetProjection())

# 写入计算后的数据
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(A)

# 设置无数据值
if nodata is not None:
    out_band.SetNoDataValue(nodata)

# 清理
out_band.FlushCache()
out_ds = None


In [8]:

# 打开原始栅格文件
input_raster = os.path.join("data", "input") # 修改为你的输入文件路径
ds = gdal.Open(input_raster)
band = ds.GetRasterBand(1)
nodata = band.GetNoDataValue()

# 读取栅格数据
data = band.ReadAsArray()

# 应用公式，只对非空值进行计算
# 注意：这里假设无数据值已经被设置为了np.nan或者一个特定的无数据标记
# 如果无数据值不是np.nan，你需要先将它们转换为np.nan或者一个适当的值
with np.errstate(divide='ignore', invalid='ignore'):  # 忽略可能的数学错误
    A = np.where(data == nodata, nodata, 34.4237 - 7.8914 * np.log(81.6593 * data ** (-1.3458) - 1))

# 创建新栅格
driver = gdal.GetDriverByName('GTiff')
output_raster = os.path.join("data", "output")  # 修改为你的输出文件路径
out_ds = driver.Create(output_raster, ds.RasterXSize, ds.RasterYSize, 1, band.DataType)
out_ds.SetGeoTransform(ds.GetGeoTransform())
out_ds.SetProjection(ds.GetProjection())

# 写入计算后的数据
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(A)

# 设置无数据值
if nodata is not None:
    out_band.SetNoDataValue(nodata)

# 清理
out_band.FlushCache()
out_ds = None

In [16]:


# 打开栅格A和栅格B
ds_a = gdal.Open(os.path.join("data", "input"))
ds_b = gdal.Open(os.path.join("data", "input"))

# 获取栅格A的尺寸
x_size_a = ds_a.RasterXSize
y_size_a = ds_a.RasterYSize

# 获取栅格B的尺寸
x_size_b = ds_b.RasterXSize
y_size_b = ds_b.RasterYSize

# 读取栅格B的数据
band_b = ds_b.GetRasterBand(1)
array_b = band_b.ReadAsArray()

# 创建一个新的数组，尺寸与栅格A相同，初始化为0
aligned_array_b = np.zeros((y_size_a, x_size_a), dtype=np.float64)

# 将栅格B的数据复制到新数组的相应位置
# 注意：这里假设栅格B位于栅格A的左上角，如果不是这样，你可能需要调整偏移量
aligned_array_b[:y_size_b, :x_size_b] = array_b

# 创建一个新的栅格文件，用于保存对齐后的栅格B数据
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(os.path.join("data", "output"), x_size_a, y_size_a, 1, band_b.DataType)
out_ds.SetGeoTransform(ds_a.GetGeoTransform())  # 使用栅格A的地理变换
out_ds.SetProjection(ds_a.GetProjection())  # 使用栅格A的投影
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(aligned_array_b)
out_band.SetNoDataValue(0)  # 设置空值为0
out_band.FlushCache()

# 清理
out_band = None
out_ds = None
band_b = None
ds_a = None
ds_b = None

print('B has been saved')


对齐后的栅格B已保存。


In [18]:


# 创建新的栅格数组
new_array = np.where(array_a == 0, array_b, array_a)

# 创建新的栅格文件以保存合并后的结果
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(os.path.join("data", "output"), ds_a.RasterXSize, ds_a.RasterYSize, 1, band_a.DataType)
out_ds.SetGeoTransform(ds_a.GetGeoTransform())
out_ds.SetProjection(ds_a.GetProjection())
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(new_array)
out_band.FlushCache()

# 清理
out_band = None
out_ds = None
band_a = None
band_b = None
ds_a = None
ds_b = None

print('saved as: merged.tif')


合并后的栅格数据已保存为: merged.tif
